## Setup

In [95]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2 as pg2
%matplotlib inline

## Import and explore

In [10]:
df = pd.read_csv('data/superchargers.csv', encoding='iso-8859-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 976 entries, 0 to 975
Data columns (total 13 columns):
Supercharger        976 non-null object
Street Address ?    0 non-null float64
City                976 non-null object
State               606 non-null object
Zip                 733 non-null object
Country             976 non-null object
Stalls              976 non-null int64
GPS                 976 non-null object
Elev.(m)            976 non-null object
Status              976 non-null object
Open Date           0 non-null float64
Tesla               976 non-null object
Discuss             976 non-null object
dtypes: float64(2), int64(1), object(10)
memory usage: 99.2+ KB


In [17]:
df.head()

,Supercharger,Street Address ?,City,State,Zip,Country,Stalls,GPS,Elev.(m),Status,Open Date,Tesla,Discuss
0,"Newark, DE",NaN,Newark,DE,19725,USA,12,"39.6635, -75.69017",18,Open,NaN,SC,thread
1,"Girona-Hotel ibis, Spain",NaN,Girona,NaN,NaN,Spain,4,"41.998411, 2.817295",83,Open,NaN,SC,thread
2,"Hangzhou-Binjiang, China",NaN,Hangzhou,Zhejiang,310051,China,4,"30.178795, 120.218038",8,Open,NaN,SC,forum
3,"Price, UT",NaN,Price,UT,84501,USA,8,"39.600731, -110.831701","1,207",Open,NaN,SC,thread
4,"Ödeshög, Sweden",NaN,Ödeshög,NaN,NaN,Sweden,8,"58.227074, 14.668253",140,Open,NaN,SC,forum


In [19]:
df['Discuss'].value_counts()

thread    608
forum     368
Name: Discuss, dtype: int64

In [20]:
df['Tesla'].value_counts()

SC    976
Name: Tesla, dtype: int64

In [25]:
df['Status'].value_counts()

Open            907
Construction     43
Permit           26
Name: Status, dtype: int64

## Clean-up
1. Remove columns: `Discuss`, `Tesla`, `Open Date`, `Street Address`
1. Split GPS coordinates
1. Convert `Elev.(m)` to numeric type

In [156]:
# Remove columns (had to use df2.columns[1] because string col name wasn't working)
df2 = df.copy()
df2.drop(['Discuss', 'Tesla', 'Open Date', df2.columns[1]], axis=1, inplace=True)
df2.head()

,Supercharger,City,State,Zip,Country,Stalls,GPS,Elev.(m),Status
0,"Newark, DE",Newark,DE,19725,USA,12,"39.6635, -75.69017",18,Open
1,"Girona-Hotel ibis, Spain",Girona,NaN,NaN,Spain,4,"41.998411, 2.817295",83,Open
2,"Hangzhou-Binjiang, China",Hangzhou,Zhejiang,310051,China,4,"30.178795, 120.218038",8,Open
3,"Price, UT",Price,UT,84501,USA,8,"39.600731, -110.831701","1,207",Open
4,"Ödeshög, Sweden",Ödeshög,NaN,NaN,Sweden,8,"58.227074, 14.668253",140,Open


In [158]:
# Split coordinates
gps_coords = df2['GPS'].str.split(', ', expand=True)
gps_coords.columns = ['Lat', 'Lon']
gps_coords['Lat'] = pd.to_numeric(gps_coords['Lat'])
gps_coords['Lon'] = pd.to_numeric(gps_coords['Lon'])

# Join back to df
df3 = df2.join(gps_coords)

# Clean-up
df3.drop('GPS', axis=1, inplace=True)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 976 entries, 0 to 975
Data columns (total 10 columns):
Supercharger    976 non-null object
City            976 non-null object
State           606 non-null object
Zip             733 non-null object
Country         976 non-null object
Stalls          976 non-null int64
Elev.(m)        976 non-null object
Status          976 non-null object
Lat             976 non-null float64
Lon             976 non-null float64
dtypes: float64(2), int64(1), object(7)
memory usage: 76.3+ KB


In [159]:
# Convert Elev.(m) data type
df4 = df3.copy()
df4['Elev.(m)'] = pd.to_numeric(df3['Elev.(m)'].str.replace(',', ''))
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 976 entries, 0 to 975
Data columns (total 10 columns):
Supercharger    976 non-null object
City            976 non-null object
State           606 non-null object
Zip             733 non-null object
Country         976 non-null object
Stalls          976 non-null int64
Elev.(m)        976 non-null int64
Status          976 non-null object
Lat             976 non-null float64
Lon             976 non-null float64
dtypes: float64(2), int64(2), object(6)
memory usage: 76.3+ KB


In [160]:
df4.head()

,Supercharger,City,State,Zip,Country,Stalls,Elev.(m),Status,Lat,Lon
0,"Newark, DE",Newark,DE,19725,USA,12,18,Open,39.663500,-75.690170
1,"Girona-Hotel ibis, Spain",Girona,NaN,NaN,Spain,4,83,Open,41.998411,2.817295
2,"Hangzhou-Binjiang, China",Hangzhou,Zhejiang,310051,China,4,8,Open,30.178795,120.218038
3,"Price, UT",Price,UT,84501,USA,8,1207,Open,39.600731,-110.831701
4,"Ödeshög, Sweden",Ödeshög,NaN,NaN,Sweden,8,140,Open,58.227074,14.668253


## Write to database

Create database

In [168]:
# Connect
conn = pg2.connect(dbname='postgres', host='localhost')
conn.autocommit = True
cur = conn.cursor()

# Create database
cur.execute('DROP DATABASE IF EXISTS tesla;')
cur.execute('CREATE DATABASE tesla;')

# Close connection
conn.close()

OperationalError: database "tesla" is being accessed by other users
DETAIL:  There is 1 other session using the database.


Create and load table from dataframe

In [208]:
# Connect
conn = pg2.connect(dbname='tesla', host='localhost')
cur = conn.cursor()

# Create table
cur.execute('DROP TABLE chargers;')

ct_query = '''
CREATE TABLE IF NOT EXISTS chargers (
    supercharger varchar(100) ,
    city varchar(100) ,
    state varchar(25) ,
    zip varchar(25) ,
    country varchar(25) ,
    stalls int ,
    elevation int ,
    status varchar(25) ,
    lat float ,
    lon float );'''

cur.execute(ct_query)
conn.commit()

In [215]:
# Load table
cur.execute('TRUNCATE TABLE chargers;')

for i in range(len(df4)):
    row = list(df4.iloc[i])
    row[5] = int(row[5])
    row[6] = int(row[6])
    cur.execute('INSERT INTO chargers VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);', 
                row)

conn.commit()

# Return rows affected
cur.execute('SELECT COUNT(*) FROM chargers;')
print('Rows inserted: {}'.format(cur.fetchone()))
conn.commit()

Rows inserted: (976,)


In [219]:
cur.execute('SELECT * FROM chargers;')
conn.commit()
cur.fetchmany(1)

[('Newark, DE',
  'Newark',
  'DE',
  '19725',
  'USA',
  12,
  18,
  'Open',
  39.6635,
  -75.69017)]

In [221]:
conn.rollback()
conn.close()

InterfaceError: connection already closed

## Read table to DataFrame and plot